# Stock Returns Prediction with Supervised Learning Algorithms
***





__Objective: To build and evaluate the performance of models based on various supervised machine learning algorithms to predict stock returns.__

The dataset to be used comprise all stocks listed on America's three stock exchanges: NYSE, AMEX and NASDAQ.

New York Stock Exchange (__NYSE__): The largest American stock exchange by volume. Combined with Europe's Deutsche Boerse and Euronext exchanges, NYSE lists companies from all over the world.

American Stock Exchange (__AMEX__): Favoured by smaller companies which cannot meet NYSE's strict listing and reporting requirements.

National Association of Securities Dealers (__NASDAQ__): Unlike the other American exchanges, NASDAQ does not operate with a physical trading floor, increasing the exchange's cost efficiency. NASDAQ is traditionally heavy with tech stocks.

Conveniently, the list of tickers for all stocks listed on the three stock exchanges can be found on the NASDAQ Stock Screener: https://www.nasdaq.com/market-activity/stocks/screener and downloaded as a csv file.

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime

In [2]:
# Data downloaded from NASDAQ Stock Screener on 05Jan2023 7am EST.
screener = pd.read_csv('nasdaq_screener_1672920378111.csv')
screener.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$151.67,1.63,1.086%,4.490525e+10,United States,1999.0,1247420,Industrials,Electrical Products
1,AA,Alcoa Corporation Common Stock,$45.13,0.55,1.234%,7.985174e+09,NaN,2016.0,3523420,Industrials,Metal Fabrications
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,$10.10,-0.01,-0.099%,1.262500e+09,NaN,2021.0,1529081,Finance,Business Services
3,AACG,ATA Creativity Global American Depositary Shares,$1.27,0.01,0.794%,3.984869e+07,China,2008.0,1350,Consumer Discretionary,Service to the Health Industry
4,AACI,Armada Acquisition Corp. I Common Stock,$10.14,0.01,0.099%,2.099943e+08,United States,2021.0,7253,Industrials,Consumer Electronics/Appliances


In [3]:
screener.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7826 entries, 0 to 7825
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      7825 non-null   object 
 1   Name        7826 non-null   object 
 2   Last Sale   7826 non-null   object 
 3   Net Change  7826 non-null   float64
 4   % Change    7822 non-null   object 
 5   Market Cap  7372 non-null   float64
 6   Country     7068 non-null   object 
 7   IPO Year    4629 non-null   float64
 8   Volume      7826 non-null   int64  
 9   Sector      6924 non-null   object 
 10  Industry    6924 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 672.7+ KB


In [4]:
# To find the missing ticker symbol from the screener DataFrame 
screener[screener['Symbol'].isna()]

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
4786,NaN,Nano Labs Ltd American Depositary Shares,$1.18,0.006,0.511%,63324700.0,China,2022.0,16258,Technology,Semiconductors


In [5]:
# 'NA' is the ticker symbol for Nano Labs Ltd ADS
# To reread the CSV file into a DataFrame but to remove 'NA' from default strings to be parsed as NaN
screener = pd.read_csv('nasdaq_screener_1672920378111.csv',keep_default_na=False,
                           na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', 
                                      '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 
                                      'NULL', 'NaN', 'n/a','nan', 'null'])

In [6]:
screener.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7826 entries, 0 to 7825
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      7826 non-null   object 
 1   Name        7826 non-null   object 
 2   Last Sale   7826 non-null   object 
 3   Net Change  7826 non-null   float64
 4   % Change    7822 non-null   object 
 5   Market Cap  7372 non-null   float64
 6   Country     7068 non-null   object 
 7   IPO Year    4629 non-null   float64
 8   Volume      7826 non-null   int64  
 9   Sector      6924 non-null   object 
 10  Industry    6924 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 672.7+ KB


In [7]:
screener['Symbol'].nunique()

7826

In [8]:
tickers = screener['Symbol'].tolist()

In [2]:
# import seaborn and matplotlib for charting and data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

In [3]:
#import plotly and cufflinks for interactive visualisations
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

In [13]:
# To determine an appropriate time period for calculating stock returns by avoiding the selection of start and end dates at peaks or troughs of the economic cycle. 
# To do this, a historical price chart for the NYSE Composite will be plotted.
# The NYSE Composite is a stock market index covering all common stock listed on the New York Stock Exchange, including American depositary receipts, real estate investment trusts, tracking stocks, and foreign listings.
NYA_historical_price = yf.download('NYA',end='2022-12-31')

[*********************100%***********************]  1 of 1 completed


In [14]:
NYA_historical_price

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1970-01-23 00:00:00-05:00,527.950012,527.950012,527.950012,527.950012,527.950012,0
1970-01-30 00:00:00-05:00,502.679993,502.679993,502.679993,502.679993,502.679993,0
1970-02-24 00:00:00-05:00,520.229980,520.229980,520.229980,520.229980,520.229980,0
1970-02-25 00:00:00-05:00,528.369995,528.369995,528.369995,528.369995,528.369995,0
1970-02-26 00:00:00-05:00,525.830017,525.830017,525.830017,525.830017,525.830017,0
...,...,...,...,...,...,...
2022-02-24 00:00:00-05:00,15996.000000,15996.000000,15996.000000,15996.000000,15996.000000,0
2022-02-25 00:00:00-05:00,16428.000000,16428.000000,16428.000000,16428.000000,16428.000000,0
2022-02-28 00:00:00-05:00,16313.900391,16313.900391,16313.900391,16313.900391,16313.900391,0


In [15]:
NYA_historical_price.iplot(y='Adj Close', 
                           title='NYSE COMPOSITE (^NYA) Price Chart', 
                           yTitle='Adjusted Close Price (USD)')

From the historical price performance of the NYSE Composite:
<br> Beginning of Jan-2010 is an appropriate start date as it is post the Global Financial Crisis in 2008. 
<br> Beginning of Jan-2020 can be selected as the end date for calculating historical stock returns as it is before Covid-19, a black swan event, affected financial markets.
<br> Therefore, for our machine learning algorithms, <u>historical 10-year stock returns from beginning of Jan-2010 to beginning of Jan-2020 will be used as the learning label.<u>

In [22]:
# The first trading day of 2010 is 4-Jan-2010.
start_price = yf.download(tickers, start='2010-01-04', end='2010-01-05')

[*********************100%***********************]  7826 of 7826 completed

4874 Failed downloads:
- HPP^C: No timezone found, symbol may be delisted
- DH: Data doesn't exist for startDate = 1262581200, endDate = 1262667600
- ENTA: Data doesn't exist for startDate = 1262581200, endDate = 1262667600
- AQST: Data doesn't exist for startDate = 1262581200, endDate = 1262667600
- CHPT: Data doesn't exist for startDate = 1262581200, endDate = 1262667600
- GRAY: Data doesn't exist for startDate = 1262581200, endDate = 1262667600
- RENN: Data doesn't exist for startDate = 1262581200, endDate = 1262667600
- AMBO: Data doesn't exist for startDate = 1262581200, endDate = 1262667600
- RCA: Data doesn't exist for startDate = 1262581200, endDate = 1262667600
- PLMIU: Data doesn't exist for startDate = 1262581200, endDate = 1262667600
- AGM^E: No timezone found, symbol may be delisted
- FSRX: Data doesn't exist for startDate = 1262581200, endDate = 1262667600
- PYPD: Data doesn't exist for startDate 

In [28]:
start_price.transpose().to_csv('start_price.csv')

In [24]:
# The first trading day of 2020 is 2-Jan-2020.
end_price = yf.download(tickers, start='2020-01-02', end='2020-01-03')

[*********************100%***********************]  7826 of 7826 completed

2949 Failed downloads:
- PYPD: Data doesn't exist for startDate = 1577941200, endDate = 1578027600
- ROC: Data doesn't exist for startDate = 1577941200, endDate = 1578027600
- DH: Data doesn't exist for startDate = 1577941200, endDate = 1578027600
- HPP^C: No timezone found, symbol may be delisted
- GRAY: Data doesn't exist for startDate = 1577941200, endDate = 1578027600
- AZ: Data doesn't exist for startDate = 1577941200, endDate = 1578027600
- SQZ: Data doesn't exist for startDate = 1577941200, endDate = 1578027600
- BRSHW: Period 'max' is invalid, must be one of ['1d', '5d']
- BUR: Data doesn't exist for startDate = 1577941200, endDate = 1578027600
- CTA^B: No timezone found, symbol may be delisted
- FINMW: Period 'max' is invalid, must be one of ['1d', '5d']
- APG: Data doesn't exist for startDate = 1577941200, endDate = 1578027600
- FOUR: Data doesn't exist for startDate = 1577941200, endDate = 1578027600

In [29]:
end_price.transpose().to_csv('end_price.csv')

In [16]:
# The historical price data will be loaded from locally stored csv files for convenience.
start_price = pd.read_csv('start_price.csv', names=['Metric','Symbol','Price at 04-Jan-2010'], header=0, keep_default_na=False,
                           na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', 
                                      '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 
                                      'NULL', 'NaN', 'n/a','nan', 'null'])
end_price = pd.read_csv('end_price.csv', names=['Metric','Symbol','Price at 02-Jan-2020'], header=0, keep_default_na=False,
                           na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', 
                                      '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 
                                      'NULL', 'NaN', 'n/a','nan', 'null'])

In [17]:
start_price

,Metric,Symbol,Price at 04-Jan-2010
0,Adj Close,A,20.434929
1,Adj Close,AA,36.938271
2,Adj Close,AAC,NaN
3,Adj Close,AACG,0.047792
4,Adj Close,AACI,NaN
...,...,...,...
46951,Volume,ZVSA,NaN
46952,Volume,ZWS,NaN
46953,Volume,ZYME,NaN
46954,Volume,ZYNE,NaN


In [18]:
end_price

,Metric,Symbol,Price at 02-Jan-2020
0,Adj Close,A,84.799675
1,Adj Close,AA,21.373323
2,Adj Close,AAC,NaN
3,Adj Close,AACG,1.350000
4,Adj Close,AACI,NaN
...,...,...,...
46951,Volume,ZVSA,NaN
46952,Volume,ZWS,809900.000000
46953,Volume,ZYME,301100.000000
46954,Volume,ZYNE,656100.000000


In [19]:
start_price['Metric'].unique()

array(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'],
      dtype=object)

In [20]:
end_price['Metric'].unique()

array(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'],
      dtype=object)

Adjusted close price will be used as it is a more accurate measure of stock returns as it accounts for corporate actions such as stock splits, dividends and new stock offerings.

In [21]:
start_price = start_price[start_price['Metric']=='Adj Close'].drop('Metric', axis=1)
end_price = end_price[end_price['Metric']=='Adj Close'].drop('Metric', axis=1)

In [23]:
start_price

,Symbol,Price at 04-Jan-2010
0,A,20.434929
1,AA,36.938271
2,AAC,NaN
3,AACG,0.047792
4,AACI,NaN
...,...,...
7821,ZVSA,NaN
7822,ZWS,NaN
7823,ZYME,NaN
7824,ZYNE,NaN


In [24]:
end_price

,Symbol,Price at 02-Jan-2020
0,A,84.799675
1,AA,21.373323
2,AAC,NaN
3,AACG,1.350000
4,AACI,NaN
...,...,...
7821,ZVSA,NaN
7822,ZWS,32.840000
7823,ZYME,47.590000
7824,ZYNE,5.880000


In [25]:
returns = pd.merge(left=start_price, right=end_price, on='Symbol').dropna()

In [26]:
returns

,Symbol,Price at 04-Jan-2010,Price at 02-Jan-2020
0,A,20.434929,84.799675
1,AA,36.938271,21.373323
3,AACG,0.047792,1.350000
7,AAIC,3.799236,5.540000
11,AAL,4.496876,28.982893
...,...,...,...
7803,ZIVO,0.280000,0.160000
7807,ZNH,13.289250,35.099998
7816,ZTR,3.777640,9.547862
7818,ZUMZ,12.360000,34.090000


In [27]:
# To create our learning label for machine learning.
returns['Effective annual return'] = (returns['Price at 02-Jan-2020']/returns['Price at 04-Jan-2010'])**(1/10)-1

In [28]:
returns

,Symbol,Price at 04-Jan-2010,Price at 02-Jan-2020,Effective annual return
0,A,20.434929,84.799675,0.152928
1,AA,36.938271,21.373323,-0.053241
3,AACG,0.047792,1.350000,0.396683
7,AAIC,3.799236,5.540000,0.038440
11,AAL,4.496876,28.982893,0.204823
...,...,...,...,...
7803,ZIVO,0.280000,0.160000,-0.054425
7807,ZNH,13.289250,35.099998,0.101998
7816,ZTR,3.777640,9.547862,0.097156
7818,ZUMZ,12.360000,34.090000,0.106779


In [29]:
stockinfo = pd.merge(left=screener, right=returns, on='Symbol')

In [30]:
stockinfo

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,Price at 04-Jan-2010,Price at 02-Jan-2020,Effective annual return
0,A,Agilent Technologies Inc. Common Stock,$151.67,1.630,1.086%,4.490525e+10,United States,1999.0,1247420,Industrials,Electrical Products,20.434929,84.799675,0.152928
1,AA,Alcoa Corporation Common Stock,$45.13,0.550,1.234%,7.985174e+09,NaN,2016.0,3523420,Industrials,Metal Fabrications,36.938271,21.373323,-0.053241
2,AACG,ATA Creativity Global American Depositary Shares,$1.27,0.010,0.794%,3.984869e+07,China,2008.0,1350,Consumer Discretionary,Service to the Health Industry,0.047792,1.350000,0.396683
3,AAIC,Arlington Asset Investment Corp Class A (new),$2.96,0.010,0.339%,8.396537e+07,United States,NaN,26879,Real Estate,Real Estate Investment Trusts,3.799236,5.540000,0.038440
4,AAL,American Airlines Group Inc. Common Stock,$13.59,0.850,6.672%,8.832153e+09,United States,NaN,34359051,Consumer Discretionary,Air Freight/Delivery Services,4.496876,28.982893,0.204823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,ZIVO,Zivo Bioscience Inc. Common Stock,$2.49,0.000,0.00%,2.345495e+07,United States,NaN,2633,Health Care,Biotechnology: Pharmaceutical Preparations,0.280000,0.160000,-0.054425
2948,ZNH,China Southern Airlines Company Limited Common...,$35.21,1.390,4.11%,1.276073e+10,China,1997.0,15398,Consumer Discretionary,Air Freight/Delivery Services,13.289250,35.099998,0.101998
2949,ZTR,Virtus Total Return Fund Inc.,$6.70,0.050,0.752%,3.225677e+08,United States,1988.0,243246,Finance,Investment Managers,3.777640,9.547862,0.097156
2950,ZUMZ,Zumiez Inc. Common Stock,$22.575,0.565,2.567%,4.400039e+08,United States,2005.0,428184,NaN,NaN,12.360000,34.090000,0.106779


In [31]:
stockinfo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2952 entries, 0 to 2951
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Symbol                   2952 non-null   object 
 1   Name                     2952 non-null   object 
 2   Last Sale                2952 non-null   object 
 3   Net Change               2952 non-null   float64
 4   % Change                 2952 non-null   object 
 5   Market Cap               2952 non-null   float64
 6   Country                  2914 non-null   object 
 7   IPO Year                 1127 non-null   float64
 8   Volume                   2952 non-null   int64  
 9   Sector                   2778 non-null   object 
 10  Industry                 2778 non-null   object 
 11  Price at 04-Jan-2010     2952 non-null   float64
 12  Price at 02-Jan-2020     2952 non-null   float64
 13  Effective annual return  2950 non-null   float64
dtypes: float64(6), int64(1),

In [32]:
stockinfo[stockinfo['Effective annual return'].isnull()]

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,Price at 04-Jan-2010,Price at 02-Jan-2020,Effective annual return
865,ELA,Envela Corporation Common Stock,$5.22,-0.04,-0.76%,140545269.0,United States,NaN,39409,Consumer Discretionary,Consumer Specialties,-0.000002,1.380000,NaN
1426,ITRN,Ituran Location and Control Ltd. Ordinary Shares,$21.00,-0.20,-0.943%,425611809.0,Israel,2005.0,14127,Technology,Electronic Components,-0.417380,24.200232,NaN


In [33]:
# To drop the two stocks with negative adjusted close price as at 04-Jan-2010.
stockinfo = stockinfo[stockinfo['Effective annual return'].notna()]

In [34]:
stockinfo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2950 entries, 0 to 2951
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Symbol                   2950 non-null   object 
 1   Name                     2950 non-null   object 
 2   Last Sale                2950 non-null   object 
 3   Net Change               2950 non-null   float64
 4   % Change                 2950 non-null   object 
 5   Market Cap               2950 non-null   float64
 6   Country                  2912 non-null   object 
 7   IPO Year                 1126 non-null   float64
 8   Volume                   2950 non-null   int64  
 9   Sector                   2776 non-null   object 
 10  Industry                 2776 non-null   object 
 11  Price at 04-Jan-2010     2950 non-null   float64
 12  Price at 02-Jan-2020     2950 non-null   float64
 13  Effective annual return  2950 non-null   float64
dtypes: float64(6), int64(1),

In [35]:
stockinfo['Country'].nunique()

46

In [36]:
stockinfo['Industry'].nunique()

148

In [37]:
stockinfo['Sector'].nunique()

12

In [38]:
stockinfo['Sector'].value_counts()

Finance                   790
Consumer Discretionary    518
Health Care               342
Industrials               308
Technology                298
Real Estate               125
Energy                    107
Utilities                 107
Consumer Staples           80
Telecommunications         48
Basic Materials            37
Miscellaneous              16
Name: Sector, dtype: int64

For the machine learning models, only features from Jan-2010 will be used to predict the stock returns (effective annual returns). Therefore, columns 'Last Sale', 'Net Change', '% Change', 'Market Cap' and 'Volume' are to be removed as they were taken from the NASDAQ Stock Screener on 05Jan2023 7am EST. <br> Column 'Industry' will be dropped as there are too many unique values and furthermore, column 'Sector' will be retained which captures similar information, though not as granular.<br> Stock prices will be dropped as they were used to create the effective annual returns learning label.

In [39]:
stockinfo = stockinfo.drop(['Last Sale','Net Change','% Change','Market Cap','Volume','Industry','Price at 04-Jan-2010','Price at 02-Jan-2020'], axis=1)

In [54]:
stockinfo

,Symbol,Name,Country,IPO Year,Sector,Effective annual return
0,A,Agilent Technologies Inc. Common Stock,United States,1999.0,Industrials,0.152928
1,AA,Alcoa Corporation Common Stock,NaN,2016.0,Industrials,-0.053241
2,AACG,ATA Creativity Global American Depositary Shares,China,2008.0,Consumer Discretionary,0.396683
3,AAIC,Arlington Asset Investment Corp Class A (new),United States,NaN,Real Estate,0.038440
4,AAL,American Airlines Group Inc. Common Stock,United States,NaN,Consumer Discretionary,0.204823
...,...,...,...,...,...,...
2947,ZIVO,Zivo Bioscience Inc. Common Stock,United States,NaN,Health Care,-0.054425
2948,ZNH,China Southern Airlines Company Limited Common...,China,1997.0,Consumer Discretionary,0.101998
2949,ZTR,Virtus Total Return Fund Inc.,United States,1988.0,Finance,0.097156
2950,ZUMZ,Zumiez Inc. Common Stock,United States,2005.0,NaN,0.106779


In [55]:
stockinfo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2950 entries, 0 to 2951
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Symbol                   2950 non-null   object 
 1   Name                     2950 non-null   object 
 2   Country                  2912 non-null   object 
 3   IPO Year                 1126 non-null   float64
 4   Sector                   2776 non-null   object 
 5   Effective annual return  2950 non-null   float64
dtypes: float64(2), object(4)
memory usage: 161.3+ KB


In [48]:
# To faciliate downloading of feature data from Capital IQ, we will download longName from yahooquery for the stocks. The names downloaded from NASDAQ stock screener include descriptions like 'Common Stock'.
tickers = stockinfo['Symbol'].tolist()

In [49]:
from yahooquery import Ticker

tickernames = pd.DataFrame.from_dict(Ticker(tickers).price).loc['longName']

In [57]:
tickernames

A                    Agilent Technologies, Inc.
AA                            Alcoa Corporation
AACG                      ATA Creativity Global
AAIC           Arlington Asset Investment Corp.
AAL                American Airlines Group Inc.
                         ...                   
ZIVO                      ZIVO Bioscience, Inc.
ZNH     China Southern Airlines Company Limited
ZTR               Virtus Total Return Fund Inc.
ZUMZ                                Zumiez Inc.
ZYXI                                Zynex, Inc.
Name: longName, Length: 2950, dtype: object

In [58]:
stockinfo = tickernames.reset_index().rename(columns={'index':'Symbol'}).merge(stockinfo, on='Symbol')

In [60]:
stockinfo

,Symbol,longName,Name,Country,IPO Year,Sector,Effective annual return
0,A,"Agilent Technologies, Inc.",Agilent Technologies Inc. Common Stock,United States,1999.0,Industrials,0.152928
1,AA,Alcoa Corporation,Alcoa Corporation Common Stock,NaN,2016.0,Industrials,-0.053241
2,AACG,ATA Creativity Global,ATA Creativity Global American Depositary Shares,China,2008.0,Consumer Discretionary,0.396683
3,AAIC,Arlington Asset Investment Corp.,Arlington Asset Investment Corp Class A (new),United States,NaN,Real Estate,0.038440
4,AAL,American Airlines Group Inc.,American Airlines Group Inc. Common Stock,United States,NaN,Consumer Discretionary,0.204823
...,...,...,...,...,...,...,...
2945,ZIVO,"ZIVO Bioscience, Inc.",Zivo Bioscience Inc. Common Stock,United States,NaN,Health Care,-0.054425
2946,ZNH,China Southern Airlines Company Limited,China Southern Airlines Company Limited Common...,China,1997.0,Consumer Discretionary,0.101998
2947,ZTR,Virtus Total Return Fund Inc.,Virtus Total Return Fund Inc.,United States,1988.0,Finance,0.097156
2948,ZUMZ,Zumiez Inc.,Zumiez Inc. Common Stock,United States,2005.0,NaN,0.106779


In [62]:
stockinfo.to_csv('Stockinfo with longName.csv')

In [3]:
# To import csv file containing fundamental data downloaded from S&P Capital IQ.
stockdata = pd.read_csv('Stock data.csv', encoding='cp1252', index_col=0)

In [4]:
stockdata.head()

,Symbol,longName,Name,Country,IPO Year,Sector,Effective annual return,CIQ ID,Ticker,Country.1,...,IQ_CASH_ST_INVEST,IQ_NPPE,IQ_GW,IQ_RE,IQ_TBV,IQ_NET_DEBT,IQ_CONTINGENT_LIABILITIES,IQ_CASH_OPER,IQ_CASH_INVEST,IQ_NET_CHANGE
0,A,"Agilent Technologies, Inc.",Agilent Technologies Inc. Common Stock,United States,1999.0,Industrials,0.152928,IQ154924,NYSE:A,United States,...,2493.00000,845.00000,655.0000,2760.00000,1684.00000,412.00000,0.0,408.00000,-14.00000,1074.00000
1,AA,Alcoa Corporation,Alcoa Corporation Common Stock,NaN,2016.0,Industrials,-0.053241,IQ369991357,NYSE:AA,United States,...,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000
2,AACG,ATA Creativity Global,ATA Creativity Global American Depositary Shares,China,2008.0,Consumer Discretionary,0.396683,IQ39920439,NasdaqGM:AACG,China,...,45.43238,3.07086,3.4272,-13.47624,48.40566,-45.43238,0.0,4.61462,-5.12896,-3.17418
3,AAIC,Arlington Asset Investment Corp.,Arlington Asset Investment Corp Class A (new),United States,NaN,Real Estate,0.038440,IQ20323,NYSE:AAIC,United States,...,10.12300,0.00000,0.0000,-1364.47600,150.01300,133.56400,0.0,-77.61100,753.33100,-244.53000
4,AAL,American Airlines Group Inc.,American Airlines Group Inc. Common Stock,United States,NaN,Consumer Discretionary,0.204823,IQ168569,NasdaqGS:AAL,United States,...,4399.00000,14837.00000,0.0000,-5136.00000,-4477.00000,7298.00000,0.0,930.00000,-2723.00000,-38.00000


In [5]:
stockdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2950 entries, 0 to 2949
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Symbol                          2950 non-null   object 
 1   longName                        2950 non-null   object 
 2   Name                            2950 non-null   object 
 3   Country                         2912 non-null   object 
 4   IPO Year                        1126 non-null   float64
 5   Sector                          2776 non-null   object 
 6   Effective annual return         2950 non-null   float64
 7   CIQ ID                          2950 non-null   object 
 8   Ticker                          2840 non-null   object 
 9   Country.1                       2950 non-null   object 
 10  Industry                        2945 non-null   object 
 11  Short Business Description      2941 non-null   object 
 12  IQ_TOTAL_REV                    29

Fundamental data were downloaded from Capital IQ to be used as features to predict effective annual return. These fundamental data are for financial year ending 2009 updated as at 01-Apr-2023 (date of retrieval of data), and are in USD (for monetary figures - using historical exchange rates) or % (for ratios). <br> <br>
The fundamental data (29 metrics in total) obtained from Capital IQ have the following definitions: <br>
**IQ_TOTAL_REV**: Revenues + Other Revenues, Total	<br>
**IQ_RETURN_ASSETS**: EBIT * (1-0.375) / Average Total Assets	<br>
**IQ_RETURN_EQUITY**:	Earnings from Continuing Operations / Average Total Equity<br>
**IQ_GROSS_MARGIN**: Gross Profit / Total Revenues <br>
**IQ_EBITDA_MARGIN**: EBITDA / Total Revenues	<br>
**IQ_NI_MARGIN**: Net Income / Total Revenues	<br>
**IQ_AUDITOR_OPINION**: Auditor Opinion	<br>
**IQ_ASSET_TURNS**: Total Revenues / Average Total Assets	<br>
**IQ_CURRENT_RATIO**: Total Current Assets / Total Current Liabilities	<br>
**IQ_DAYS_SALES_OUT**: Average Accounts Receivable / Revenues * 360	<br>
**IQ_DAYS_INVENTORY_OUT**: Average Inventory / Cost of Goods Sold * 360	<br>
**IQ_DAYS_PAYABLE_OUT**: Average Accounts Payable / (Cost of Goods Sold - Beginning Inventory + Ending Inventory) * Number of Days in the Period	<br>
**IQ_TOTAL_DEBT_EQUITY**: Total Debt / Total Equity	<br>
**IQ_TOTAL_REV_1YR_ANN_GROWTH**: Total Revenues (t) / Total Revenues (t-1) - 1	<br>
**IQ_NI_1YR_ANN_GROWTH**: Net Income (t) / Net Income (t-1) -1	<br>
**IQ_CFO_1YR_ANN_GROWTH**: Cash from Operations (t) / Cash from Operations (t-1) -1	<br>
**IQ_TOTAL_ASSETS_1YR_ANN_GROWTH**: Total Assets (t) / Total Assets (t-1) -1	<br>
**IQ_EBITDA**: EBITDA	<br>
**IQ_PAYOUT_RATIO**: (Common Dividends Paid + Preferred Dividends Paid) / Net Income	<br>
**IQ_CASH_ST_INVEST**: Cash and Equivalents + Short Term Investments + Trading Asset Securities	<br>
**IQ_NPPE**: Gross Property Plant and Equipment + Accumulated Depreciation	<br>
**IQ_GW**: Goodwill	<br>
**IQ_RE**: Retained Earnings	<br>
**IQ_TBV**: Book Value of Common Equity - Goodwill = Other Intangibles - Fin. Div. Goodwill - Fin. Div. Other Intangibles	<br>
**IQ_NET_DEBT**: Total Debt - Total Cash and Short Term Investments - Long Term Marketable Securities	<br>
**IQ_CONTINGENT_LIABILITIES**: Contingent Liabilites <br>
**IQ_CASH_OPER**: Cash from Operations <br>
**IQ_CASH_INVEST**: Cash from Investing <br>
**IQ_NET_CHANGE**: Cash from Operations + Cash from Investing + Cash from Financing + Foreign Exchange Rate Adjustments + Misc. Cash Flow Adjustments